In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
%%time 
df_train = pd.read_pickle('../input/feats/df_trn_feat0.pkl')

Wall time: 116 ms


In [3]:
df_train.columns

Index(['uidx', 'iidx', 'iid', 'region_city_label', 'tit_len', 'desc_len',
       'activation_date', 'month', 'day', 'weekday', 'param_1', 'param_2',
       'param_3', 'user_type', 'parent_category_name', 'price',
       'category_name', 'image_top_1', 'ads_cnt_by_uid', 'ads_cnt_by_iid',
       'deal_probability'],
      dtype='object')

In [11]:
from libfm import *

Using TensorFlow backend.


In [35]:
def create_label_encoding_with_min_count(df, column, min_count=50):
    column_counts = df.groupby([column])[column].transform("count").astype(int)
    column_values = np.where(column_counts >= min_count, df[column], "")
    df[column+"_label"] = LabelEncoder().fit_transform(column_values)
    
    return df[column+"_label"]

In [45]:
df_train.columns

Index(['uidx', 'iidx', 'iid', 'region_city_label', 'tit_len', 'desc_len',
       'activation_date', 'month', 'day', 'weekday', 'param_1', 'param_2',
       'param_3', 'user_type', 'parent_category_name', 'price',
       'category_name', 'image_top_1', 'ads_cnt_by_uid', 'ads_cnt_by_iid',
       'deal_probability'],
      dtype='object')

In [47]:
import scipy.sparse as sp

In [50]:
from sklearn.decomposition import TruncatedSVD

In [48]:
uidx_sp = df_train[['uidx','iidx']]

I = df_train.uidx.as_matrix()
J = df_train.iidx.as_matrix()
V = np.ones(df_train.shape[0])

ui_trn_sp = sp.coo_matrix((V,(I,J)),dtype='uint8')

In [51]:
svd = TruncatedSVD(n_components=5, random_state=0)
uidx_svd = svd.fit_transform(ui_trn_sp)


In [64]:
df_train.uidx.unique()

array([     0,      1,      2, ..., 771766, 771767, 771768], dtype=uint64)

In [65]:
df_uidx_svd = pd.DataFrame(uidx_svd)
df_uidx_svd['uidx'] = df_train.uidx.unique()

In [67]:
df_train = df_train.merge(df_uidx_svd,how='left',on='uidx')

In [ ]:
%%time 
pd.read_

In [75]:
df_train[[0,1,2,3,4,'deal_probability']].corr()

,0,1,2,3,4,deal_probability
0,1.000000,0.508835,0.090404,0.014104,-0.009167,-0.008687
1,0.508835,1.000000,0.479784,0.138502,0.069193,-0.081228
2,0.090404,0.479784,1.000000,0.499284,0.176348,-0.059109
3,0.014104,0.138502,0.499284,1.000000,0.495880,-0.050277
4,-0.009167,0.069193,0.176348,0.495880,1.000000,-0.047595
deal_probability,-0.008687,-0.081228,-0.059109,-0.050277,-0.047595,1.000000


In [ ]:
# feats = ['uidx','region_city2_label']
# fm_data = df_train.groupby(feats).size().reset_index().rename(columns={0:'cnt'})

In [40]:
# tmp = df_train.groupby(['uidx','param_1','param_2','param_3'])['uidx'].transform(lambda x:np.random.random()) ## faster and encode it correctly!!
# df_train['region_city_label'] = create_label_encoding_with_min_count(df_train, 'region_city', min_count=1).astype('uint16')

ValueError: Length mismatch: Expected axis has 640859 elements, new values have 1503424 elements

In [38]:
df_train.params_label.value_counts()

ValueError: Length mismatch: Expected axis has 640859 elements, new values have 1503424 elements

In [12]:
feats = ['uidx','param_1','param_2','param_3']
fm_data = df_train.groupby(feats).size().reset_index().rename(columns={0:'cnt'})

In [23]:
fm_data.cnt.value_counts()

1      458205
2       46158
3       11322
4        3744
5        1591
6         794
7         416
8         273
9         192
10        123
11         83
12         69
14         59
13         56
15         45
16         35
19         29
18         28
17         27
21         20
20         18
26         15
30         15
23         14
24         14
22         13
29         11
27         10
34         10
38         10
        ...  
42          2
46          2
85          2
121         1
107         1
112         1
101         1
139         1
125         1
189         1
156         1
131         1
98          1
150         1
64          1
73          1
92          1
91          1
90          1
87          1
86          1
84          1
82          1
75          1
72          1
48          1
231         1
60          1
61          1
253         1
Name: cnt, Length: 101, dtype: int64

In [34]:
fm_data['param_1']

0             С пробегом
1            Для девочек
2          Для мальчиков
3          Для мальчиков
4         Женская одежда
5         Женская одежда
6          Для мальчиков
7          Для мальчиков
8         Женская одежда
9         Женская одежда
10        Женская одежда
11        Женская одежда
12        Женская одежда
13           Для девочек
14           Для девочек
15                  Сдам
16         Для мальчиков
17        Женская одежда
18        Женская одежда
19        Мужская одежда
20        Женская одежда
21         Для мальчиков
22        Женская одежда
23            С пробегом
24         Для мальчиков
25        Женская одежда
26                Продам
27                 Сниму
28                 Сниму
29                Продам
               ...      
523548       Для девочек
523549    Женская одежда
523550     Для мальчиков
523551    Женская одежда
523552            Продам
523553        С пробегом
523554     Для мальчиков
523555       Для девочек
523556    Женская одежда


In [26]:
f_size  = [int(fm_data[f].max()) + 1 for f in feats]
f_size

TypeError: Categorical is not ordered for operation max
you can use .as_ordered() to change the Categorical to an ordered one
